In [1]:
import seaborn
import matplotlib
import torch
import numpy as np
import pandas as pd
import yfinance as yf
import torch.nn as nn
import os
import time
import datetime
import pathlib
from torch.utils.data import DataLoader
import kan


In [26]:
data_names = ["S&P500","SSE","IBM","MSFT","PAICC"]
data_name = data_names[0]
data_path="./datasets/S&P500.csv"
print(data_path)
dataframe = pd.read_csv(data_path)
dataframe.describe()
print("shape = ",dataframe.shape)

./datasets/S&P500.csv
shape =  (5031, 7)


In [27]:
def add_Ma(dataframe):
  Ma_window=5
  for i in range(0,dataframe.shape[0]-Ma_window):
    dataframe.loc[dataframe.index[i+Ma_window],'Ma'] = np.round(((dataframe.iloc[i,4]+ dataframe.iloc[i+1,4] +dataframe.iloc[i+2,4] + dataframe.iloc[i+3,4]+ dataframe.iloc[i+4,4])/5),6)
  return dataframe[5:-5]

dataframe=add_Ma(dataframe)
dataframe.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ma
5,1999-01-11,1275.089966,1276.219971,1253.339966,1263.880005,1263.880005,818000000,1258.007983
6,1999-01-12,1263.880005,1264.449951,1238.290039,1239.510010,1239.510010,800200000,1265.163989
7,1999-01-13,1239.510010,1247.750000,1205.459961,1234.400024,1234.400024,931500000,1264.109985
8,1999-01-14,1234.400024,1236.810059,1209.540039,1212.189941,1212.189941,797200000,1256.521997
9,1999-01-15,1212.189941,1243.260010,1212.189941,1243.260010,1243.260010,798100000,1245.013989


In [28]:
import torch
from torch import nn
from torch.nn.utils.rnn import pack_sequence
from torchsummary import summary

class GeneratorModel(nn.Module):
    def __init__(self, n_sequence, n_features):
        super(GeneratorModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size=n_features, hidden_size=10, batch_first=True)
        self.batch_norm1 = nn.BatchNorm1d(10)
        self.leaky_relu = nn.LeakyReLU(0.3)
        self.dropout1 = nn.Dropout(0.3)
        
        self.lstm2 = nn.LSTM(input_size=10, hidden_size=10, batch_first=True)
        self.batch_norm2 = nn.BatchNorm1d(10)  
        self.dropout2 = nn.Dropout(0.3)
        
        self.output_dense = nn.Linear(10, n_features)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.batch_norm1(x.permute(0, 2, 1)).permute(0, 2, 1) 
        x = self.leaky_relu(x)
        x = self.dropout1(x)

        _, (x, _) = self.lstm2(x)
        x=x.permute(1, 0, 2)
        x = self.batch_norm2(x.permute(0, 2, 1)).permute(0, 2, 1)  
        x = self.leaky_relu(x)
        x = self.dropout2(x)

        x = self.output_dense(x)
        x = self.leaky_relu(x)

        return x

# Example usage
n_sequence = 5  # Sequence length
n_features = 7   # Number of features

class DiscriminatorModel(nn.Module):
    def __init__(self, n_sequence, n_features):
        super(DiscriminatorModel, self).__init__()
        input_dim = (n_sequence + 1) * n_features
        
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim, 72),
            nn.LeakyReLU(0.3),
            nn.Dropout(0.3),
            
            nn.Linear(72, 100),
            nn.LeakyReLU(0.3),
            nn.Dropout(0.3),
            
            nn.Linear(100, 10),
            nn.LeakyReLU(0.3),
            nn.Dropout(0.3),
            
            nn.Linear(10, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)
generator = GeneratorModel(n_sequence, n_features).to(torch.float64)
discriminator = DiscriminatorModel(n_sequence,n_features).to(torch.float64)



In [29]:
torch.set_printoptions(precision=8)
class Standarized_TimeseriesGenerator(torch.utils.data.Dataset):
    def __init__(self, data, targets, length, batch_size=1, stride=1):
        self.data = data
        self.targets = targets
        self.length = length
        self.batch_size = batch_size
        self.stride = stride
        
    def __getitem__(self, index):
        samples = [self.data[i:i+self.length] for i in range(index, len(self.data)-self.length, self.stride)]
        targets = [self.targets[i+self.length] for i in range(index, len(self.data)-self.length, self.stride)]
        # Pack sequences into tensor
        samples = torch.tensor(samples[0])
        targets = torch.tensor(targets[0])
        samples= samples.to(torch.float64)
        targets= targets.to(torch.float64)
        # shape : (n_batch, n_sequence, n_features)
        mean = samples.mean(dim=0)
        std = samples.std(dim=0,correction=0)
        samples = (samples - mean)/std  # standardize along each feature


        # targets = (targets - mean[..., 3])/std[..., 3]  # The close value is our target
        targets = (targets - mean)/std  # The close value is our target
        return samples, targets
    
    def __len__(self):
        return len(self.data) - self.length


In [30]:
# Create sample data
data = np.array([[i, i**2,i**3,i**4] for i in range(11)])
targets = data
print(targets.shape)

# Print shape of targets
# Calculate mean and std for data
mean = data[:-1].mean(axis=0)[None,:]
std = data[:-1].std(axis=0)[None,:]
# Create Standarized_TimeseriesGenerator instance
data_gen = Standarized_TimeseriesGenerator(data, targets,
                               length=10, batch_size=2, stride=1)
# Retrieve first batch
batch_0 = data_gen[0]
x, y = batch_0
print(np.array(x*std+mean))
print(y*std+mean)

(11, 4)
[[0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [1.000e+00 1.000e+00 1.000e+00 1.000e+00]
 [2.000e+00 4.000e+00 8.000e+00 1.600e+01]
 [3.000e+00 9.000e+00 2.700e+01 8.100e+01]
 [4.000e+00 1.600e+01 6.400e+01 2.560e+02]
 [5.000e+00 2.500e+01 1.250e+02 6.250e+02]
 [6.000e+00 3.600e+01 2.160e+02 1.296e+03]
 [7.000e+00 4.900e+01 3.430e+02 2.401e+03]
 [8.000e+00 6.400e+01 5.120e+02 4.096e+03]
 [9.000e+00 8.100e+01 7.290e+02 6.561e+03]]
tensor([[   10.00000000,   100.00000000,  1000.00000000, 10000.00000000]],
       dtype=torch.float64)


In [31]:
n_sequence = 5
n_features = 7
n_batch = 32

def get_gen_train_test(dataframe):
  data = dataframe.drop(columns='Date').to_numpy()
  #targets = data[:,3, None] #add none to have same number of dimensions as data
  targets = data
  n_samples = data.shape[0]
  train_test_split=int(n_samples*0.9)
  train_data = data[:train_test_split]
  test_data = data[train_test_split:]
  train_target = targets[:train_test_split]
  test_target = targets[train_test_split:]
  data_train = Standarized_TimeseriesGenerator(train_data, train_target,
                                length=n_sequence, 
                                stride=1)
  data_test = Standarized_TimeseriesGenerator(test_data, test_target,
                                length=n_sequence, 
                                stride=1)
  
  train_loader = DataLoader(data_train, batch_size=n_batch, shuffle=True)
  test_loader = DataLoader(data_test, batch_size=n_batch, shuffle=False)
  return train_loader, test_loader

data_gen_train, data_gen_test = get_gen_train_test(dataframe)

In [32]:
# test on data
data = dataframe.drop(columns='Date').to_numpy()
targets = data

x_gen, y_gen = Standarized_TimeseriesGenerator(data, targets,
                               length=5, 
                               stride=1, batch_size=1)[0]


x = data[None, :5,:]
mean = x.mean(axis=1)
std = x.std(axis=1)
x = (x - mean)/std
y = (data[5] - mean)/std
print(torch.tensor(x)-x_gen)
print(torch.tensor(y)-y_gen)
print(x_gen.shape)
print(y_gen.reshape(1,7).shape)

tensor([[[-1.33226763e-15,  3.33066907e-15, -2.44249065e-15, -1.33226763e-15,
          -1.33226763e-15,  0.00000000e+00,  5.20417043e-16],
         [-8.88178420e-16,  1.66533454e-15, -1.22124533e-15, -4.16333634e-17,
          -4.16333634e-17,  0.00000000e+00,  1.57651669e-14],
         [ 2.77555756e-16, -8.88178420e-16,  1.55431223e-15,  2.22044605e-16,
           2.22044605e-16,  0.00000000e+00,  1.35447209e-14],
         [ 4.99600361e-16, -2.66453526e-15,  1.22124533e-15,  1.33226763e-15,
           1.33226763e-15,  0.00000000e+00, -2.63677968e-15],
         [ 1.55431223e-15, -1.66533454e-15,  9.99200722e-16, -2.22044605e-16,
          -2.22044605e-16,  0.00000000e+00, -2.70894418e-14]]],
       dtype=torch.float64)
tensor([[ 8.32667268e-17, -6.24500451e-17, -8.88178420e-16, -7.77156117e-16,
         -7.77156117e-16,  0.00000000e+00, -4.04121181e-14]],
       dtype=torch.float64)
torch.Size([5, 7])
torch.Size([1, 7])


In [36]:

a1 = 0.01
a2 = 0.99
learning_rate1 = 0.0001
learning_rate2 = 0.0001

loss_fn = nn.BCELoss()
mse_fn = nn.MSELoss()

def descriminator_loss(real_output, fake_output):
    real_loss = loss_fn(real_output, torch.ones_like(real_output))
    fake_loss = loss_fn(fake_output, torch.zeros_like(fake_output))
    return real_loss + fake_loss
def generator_loss(x,y,fake_output):
    loss = loss_fn(fake_output, torch.ones_like(fake_output))
    mse_loss = mse_fn(x.reshape(-1,1,7),y.reshape(-1,1,7))
    return a1*mse_loss + a2*loss , mse_loss


In [37]:
import matplotlib.pyplot as plt
import torch
from torchvision.utils import make_grid
import tqdm
import time

torch.manual_seed(8)

def train_GAN(G, D, optim_G, optim_D, loss_f, train_loader, test_loader, num_epochs, device):
    test_size = len(test_loader)
    best = np.inf 
    for epoch in range(num_epochs):
        for i,data in enumerate(train_loader):
            if i<140:
                generator.train()
                discriminator.train()   
                starting_time = time.time()
                sequence , target = data
                sequence = sequence.to(device)
                target = target.to(device)
                generator_fake = G(sequence)
                true_seq = torch.concat((sequence,target.reshape(-1,1,7)),dim=1)
                fake_seq = torch.concat((sequence,generator_fake),dim=1)
                # ========================
                #   Train Discriminator
                # ========================
                # train with real data
                
                prediction = D(true_seq)

                # train with fake data
                
                fake_predection = D(fake_seq)
                d_loss = descriminator_loss(prediction, fake_predection)
                # update D
                
                D.zero_grad()
                d_loss.backward()
                optim_D.step()

                # ========================
                #   Train Generator
                # ========================
                # train with fake data  
                        
                sequence , target = data
                sequence = sequence.to(device)
                target = target.to(device)
                generator_fake = G(sequence)
                fake_seq = torch.concat((sequence,generator_fake),dim=1)
                fake_predection = D(fake_seq)
                g_loss, mse_loss = generator_loss(generator_fake,target,fake_predection)
                # update G
                G.zero_grad()
                g_loss.backward()
                optim_G.step()   
        print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f \tMSE_loss: %.4f \tTime: %.4f'% (epoch, num_epochs, i, len(train_loader), d_loss.item(), g_loss.item(),mse_loss.item(),time.time()-starting_time))
        #evaluate 
        dis_loss = 0
        gen_loss= 0
        mse_loss = 0
        diss_losses = []
        gen_losses = []
        mse_losses = []

        for i,data in enumerate(test_loader):
            generator.eval()
            discriminator.eval()
            with torch.no_grad():
                sequence , target = data
                sequence = sequence.to(device)
                target = target.to(device)
                generator_fake = G(sequence)
                fake_seq = torch.concat((sequence,generator_fake),dim=1)
                true_seq = torch.concat((sequence,target.reshape(-1,1,7)),dim=1)
                fake_predection = D(fake_seq)
                g_loss, mse = generator_loss(generator_fake,target,fake_predection)
                prediction = D(true_seq)
                d_loss = descriminator_loss(prediction, fake_predection)
                dis_loss += d_loss.item()/test_size
                gen_loss += g_loss.item()/test_size
                mse_loss += mse.item()/test_size
        diss_losses.append(dis_loss)
        gen_losses.append(gen_loss)
        mse_losses.append(mse_loss)

        if mse_loss < best:
            best = mse_loss
            torch.save({
                'model_state_dict': G.state_dict(),
                'optimizer_state_dict': optim_G.state_dict(),
            }, 'lstm_gan.pth') 
        print('Validation \tLoss_D: %.4f\tLoss_G: %.4f \tMSE_loss: %.4f \tBest_loss: %.4f'% (dis_loss, gen_loss,mse_loss,best))
    return diss_losses,gen_losses,mse_losses


    

epochs = 200
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = GeneratorModel(n_sequence, n_features).to(torch.float64)
discriminator = DiscriminatorModel(n_sequence,n_features).to(torch.float64)
generator , discriminator = generator.to(device), discriminator.to(device)
learning_rate1 = 0.00003
learning_rate2 = 0.00003
generator_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate1, betas=(0.5, 0.999))
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=learning_rate2, betas=(0.5, 0.999))
checkpoint_path = "./train_checkpoints"

results = train_GAN(generator, discriminator, generator_optimizer, discriminator_optimizer, loss_fn, data_gen_train, data_gen_test, epochs, device)
    

[0/200][141/142]	Loss_D: 1.3792	Loss_G: 0.7976 	MSE_loss: 4.7828 	Time: 0.0490
Validation 	Loss_D: 1.3747	Loss_G: 0.8021 	MSE_loss: 5.5139 	Best_loss: 5.5139
[1/200][141/142]	Loss_D: 1.3691	Loss_G: 0.7673 	MSE_loss: 4.8526 	Time: 0.0454
Validation 	Loss_D: 1.3615	Loss_G: 0.7864 	MSE_loss: 5.4582 	Best_loss: 5.4582
[2/200][141/142]	Loss_D: 1.3611	Loss_G: 0.7539 	MSE_loss: 3.4702 	Time: 0.0438
Validation 	Loss_D: 1.3464	Loss_G: 0.7703 	MSE_loss: 5.4213 	Best_loss: 5.4213
[3/200][141/142]	Loss_D: 1.3324	Loss_G: 0.7657 	MSE_loss: 5.1758 	Time: 0.0525
Validation 	Loss_D: 1.3291	Loss_G: 0.7605 	MSE_loss: 5.3634 	Best_loss: 5.3634
[4/200][141/142]	Loss_D: 1.3107	Loss_G: 0.7570 	MSE_loss: 5.8688 	Time: 0.0456
Validation 	Loss_D: 1.3062	Loss_G: 0.7542 	MSE_loss: 5.3105 	Best_loss: 5.3105
[5/200][141/142]	Loss_D: 1.3237	Loss_G: 0.7276 	MSE_loss: 3.3354 	Time: 0.0412
Validation 	Loss_D: 1.2789	Loss_G: 0.7512 	MSE_loss: 5.2312 	Best_loss: 5.2312
[6/200][141/142]	Loss_D: 1.2702	Loss_G: 0.7669 	MSE_

In [38]:
results = train_GAN(generator, discriminator, generator_optimizer, discriminator_optimizer, loss_fn, data_gen_train, data_gen_test, epochs, device)


[0/200][141/142]	Loss_D: 1.0274	Loss_G: 1.1362 	MSE_loss: 5.2902 	Time: 0.0493
Validation 	Loss_D: 1.2099	Loss_G: 0.9836 	MSE_loss: 3.9702 	Best_loss: 3.9702
[1/200][141/142]	Loss_D: 1.2259	Loss_G: 1.0222 	MSE_loss: 3.8514 	Time: 0.0372
Validation 	Loss_D: 1.2149	Loss_G: 0.9821 	MSE_loss: 3.9236 	Best_loss: 3.9236
[2/200][141/142]	Loss_D: 1.1886	Loss_G: 1.1685 	MSE_loss: 2.3082 	Time: 0.0510
Validation 	Loss_D: 1.2160	Loss_G: 0.9816 	MSE_loss: 3.9339 	Best_loss: 3.9236
[3/200][141/142]	Loss_D: 1.0799	Loss_G: 0.8178 	MSE_loss: 4.4767 	Time: 0.0476
Validation 	Loss_D: 1.2212	Loss_G: 0.9714 	MSE_loss: 3.9235 	Best_loss: 3.9235
[4/200][141/142]	Loss_D: 1.2099	Loss_G: 1.3141 	MSE_loss: 4.1960 	Time: 0.0454
Validation 	Loss_D: 1.2245	Loss_G: 0.9700 	MSE_loss: 3.9034 	Best_loss: 3.9034
[5/200][141/142]	Loss_D: 1.2291	Loss_G: 1.3092 	MSE_loss: 4.6321 	Time: 0.0488
Validation 	Loss_D: 1.2228	Loss_G: 0.9698 	MSE_loss: 3.8883 	Best_loss: 3.8883
[6/200][141/142]	Loss_D: 1.2782	Loss_G: 1.1091 	MSE_

In [40]:
generator.load_state_dict(torch.load('lstm_gan.pth')['model_state_dict'])
data = dataframe.drop(columns='Date').to_numpy()
targets = data
n_samples = data.shape[0]
train_test_split=int(n_samples*0.9)
train_data = data[:train_test_split]
test_data = data[train_test_split:]
train_target = targets[:train_test_split]
test_target = targets[train_test_split:]
test_data=Standarized_TimeseriesGenerator(test_data, test_target, length=n_sequence, stride=1)

mape = 0
rmse = 0
mae = 0

for i in range(0,len(test_data)-1,2):
    with torch.no_grad():
        seq = torch.concat((test_data[i][0].reshape(1,5,7),test_data[i+1][0].reshape(1,5,7)),dim=0)
        targets = torch.concat((test_data[i][1].reshape(1,1,7),test_data[i+1][1].reshape(1,1,7)),dim=0).to(device)
        pred = generator(seq.to(device))
        mape += torch.sum(torch.abs(targets[:,:,3] - pred[:,:,3] /targets[:,:,3]) )
        rmse += torch.sum((targets[:,:,3]  - pred[:,:,3] )**2)
        mae += torch.sum(torch.abs(targets[:,:,3] - pred[:,:,3] ))
        
print("MAPE = ",mape/len(test_data))
print("RMSE = ",(rmse/len(test_data))**0.5)
print("MAE = ",mae/len(test_data))


MAPE =  tensor(3.10181688, device='cuda:0', dtype=torch.float64)
RMSE =  tensor(2.02099880, device='cuda:0', dtype=torch.float64)
MAE =  tensor(1.33180923, device='cuda:0', dtype=torch.float64)
